In [1]:
# -- your name here, besure to make it a markdown.

# Assignment 4 
----

This assignment shouldn't take too long, all you need to do is apply some logic to finish off the function below. 

> NOTE: there are two opportunties for extra credit in this assighment making this assignment eligible for an extra  20 pts. 

once you've got your function(s) running simply use your function(s) to profile these datasets: 
    - churn.csv
    - bank-additional-full.csv
    - loans.csv
    
Finally, turn in an HTML version showing that your function ran on the datasets. 

In [2]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))
import warnings
warnings.filterwarnings('ignore')
# ------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline


Here i loaded the loans data into a dataframe to get my function working. 

In [3]:
#df = df[["Current_Loan_Amount", "Credit_Score", "Years_in_current_job", "Home_Ownership"]].copy()
#df.to_csv("loans_clean.csv", index=False)

In [4]:
df = pd.read_csv("../Week_4/Data/loans.csv")

loans = pd.read_csv("../Week_4/Data/loans.csv")
adult = pd.read_csv("../Week_4/Data/adult.csv")
df.head()

,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens,ID,loan_default
0,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0,20,0
1,215952.0,Short Term,739.0,1454735.0,< 1 year,Rent,Debt Consolidation,39277.75,13.9,NaN,20.0,0.0,669560.0,1021460.0,0.0,0.0,100,0
2,99999999.0,Short Term,728.0,714628.0,3 years,Rent,Debt Consolidation,11851.06,16.0,76.0,16.0,0.0,203965.0,289784.0,0.0,0.0,110,0
3,99999999.0,Short Term,740.0,776188.0,< 1 year,Own Home,Debt Consolidation,11578.22,8.5,25.0,6.0,0.0,134083.0,220220.0,0.0,0.0,130,0
4,99999999.0,Short Term,743.0,1560907.0,4 years,Rent,Debt Consolidation,17560.37,13.3,NaN,10.0,1.0,225549.0,496474.0,1.0,0.0,140,0


In [5]:
adult.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## Assignment Tasks 

your job is to finish off the profiling function. 

1. figure out how to remove scientific notation for numeric varaibles here is a article that explains several ways to do this
    - https://re-thought.com/how-to-suppress-scientific-notation-in-pandas/
    
2. create a new column called feature_type, using the following rules 
    - if feature_name equals target then feature_type = "TARGET"
    - if a feature's dtype is int or float then feature_type = "NUMERIC"
    - if a feature's dtupe is object then feature_type = "CATEGORY"

3. create a new column called feature_warning, using the following rules 
    - if nunique_pct > 0.9 then feature_warning = "EXCLUDE, GT 90% UNIQUE"
    - if dtype is int or float AND nunique <= 20 then feature_warning = "LIKELY CATEGORICAL, NUMERIC w. LOW CARDINALITY"
    - if null_pct > 0.2 then feature_warning = "NULL WARNING, GT 20% MISSING"
    - if null_pct > 0.5 then feature_warning = "EXCLUDE, GT 50% MISSING"
    - else "NO WARNING"
    

4. use your function to profile 
    - churn.csv
    - bank-additional-full.csv
    - loans.csv



In [6]:
#import modin as pd 

def df_profile(df, target):
    
    df = df.copy()
    
    numeric_features = [ c for c in df.select_dtypes(include = np.number).columns ]
    
    categorical_features = [ c for c in df.select_dtypes(include = np.object).columns ]
   
    rowcnt = len(df)
    df_s1  = df.agg(['count', 'nunique','mean','min','max']).transpose().reset_index().rename(columns={"index":"feature_name"}).round(4)
    df_s1['count'] = df_s1['count'].astype('int64')
    df_s1['nunique'] = df_s1['nunique'].astype('int64')
    df_s1["null"] = (rowcnt - df_s1["count"]).astype('int64')
    df_s1["not_null"] = rowcnt - df_s1["null"]
    df_s1["null_pct"] = df_s1["null"] / rowcnt
    df_s1["nunique_pct"] = df_s1['nunique'] / rowcnt
    dt = pd.DataFrame(df.dtypes).reset_index().rename(columns={"index":"feature_name", 0:"dtype"})
    df_stats = pd.merge(dt, df_s1, on='feature_name', how='inner')

    df_stats = df_stats.sort_values("dtype").reset_index(drop=True)
    # -- part 2 
    df_stats['feature_type'] = "UNKOWN"
    df_stats.loc[df_stats["dtype"] == object, 'feature_type'] = "CATEGORY"
    df_stats.loc[(df_stats["dtype"] == "int64") | (df_stats["dtype"] == "float64"), 'feature_type'] = "NUMERIC"
    # df_stats.loc[df_stats["dtype"] == "float64", 'feature_type'] = "NUMERIC"
    df_stats.loc[df_stats["feature_name"] == target, 'feature_type'] = "TARGET"
    
    
    # -- part 3 
    df_stats['feature_warning'] = "NO WARNING"
    df_stats.loc[df_stats["nunique_pct"] > 0.9,'feature_warning' ] = "EXCLUDE, GT 90% UNIQUE"
    df_stats.loc[(df_stats["null_pct"] > 0.2) & (df_stats["null_pct"] <= 0.5), 'feature_warning' ] = "NULL WARNING, GT 20% MISSING"
    df_stats.loc[df_stats["null_pct"] > 0.5,'feature_warning' ] = "EXCLUDE, GT 50% MISSING"
    #if null_pct > 0.2 then feature_warning = "NULL WARNING, GT 20% MISSING"
    df_stats.loc[((df_stats['dtype'] == "int64" ) | (df_stats['dtype'] == "float64" ) ) & (df_stats['nunique'] < 0.2), 'feature_warning' ] = "LIKELY CATEGORICAL, NUMERIC w. LOW CARDINALITY"
    #df_stats.loc[((df_stats['dtype'] == "int64" ) | (df_stats['dtype'] == "float64" ) ) & (df_stats['nunique'] < 0.2), 'feature_warning' ] = "LIKELY CATEGORICAL, NUMERIC w. LOW CARDINALITY"
 
    return df_stats

df_stats = df_profile(df, "loan_default")


#df_stats = df_profile(adult, "income")
df_stats

,feature_name,dtype,count,nunique,mean,min,max,null,not_null,null_pct,nunique_pct,feature_type,feature_warning
0,loan_default,int64,70478,2,2.267000e-01,0.0,1.000000e+00,0,70478,0.000000,0.000028,TARGET,NO WARNING
1,ID,int64,70478,70478,5.027010e+05,20.0,1.005120e+06,0,70478,0.000000,1.000000,NUMERIC,"EXCLUDE, GT 90% UNIQUE"
2,Tax_Liens,float64,70144,11,2.880000e-02,0.0,1.100000e+01,334,70144,0.004739,0.000156,NUMERIC,NO WARNING
3,Bankruptcies,float64,69990,8,1.175000e-01,0.0,7.000000e+00,488,69990,0.006924,0.000114,NUMERIC,NO WARNING
4,Maximum_Open_Credit,float64,70150,37594,7.718260e+05,0.0,1.539738e+09,328,70150,0.004654,0.533415,NUMERIC,NO WARNING
5,Current_Credit_Balance,float64,70152,28816,2.944997e+05,0.0,1.298696e+07,326,70152,0.004626,0.408865,NUMERIC,NO WARNING
6,Number_of_Credit_Problems,float64,70152,13,1.672000e-01,0.0,1.200000e+01,326,70152,0.004626,0.000184,NUMERIC,NO WARNING
7,Number_of_Open_Accounts,float64,70152,51,1.114820e+01,0.0,7.600000e+01,326,70152,0.004626,0.000724,NUMERIC,NO WARNING
8,Months_since_last_delinquent,float64,32937,95,3.480020e+01,0.0,9.700000e+01,37541,32937,0.532663,0.001348,NUMERIC,"EXCLUDE, GT 50% MISSING"
9,Current_Loan_Amount,float64,70152,19782,1.165346e+07,10802.0,1.000000e+08,326,70152,0.004626,0.280683,NUMERIC,NO WARNING


## Extra Credit 1 - worth 10 pts 

using your data frame and df_profile results write a function that creates a bar chart of CATEGORIAL by TARGET

use your funcion to visualize CATEORICAL data in the following datasets 
    - churn.csv
    - bank-additional-full.csv
    - loans.csv



In [12]:
def f(row):
    if row["feature_name"] == "target":
        feature_type = "TARGET"
    if row["dtype"] == 'int64' or 'float64':
        feature_type = "NUMERIC"
    elif row["dtype"] == "object":
        feature_type = "CATEGORY"
    return feature_type

df_stats['feature_type'] = df_stats.apply(f, axis=1)
df_stats

,feature_name,dtype,count,nunique,mean,min,max,null,not_null,null_pct,nunique_pct,feature_type,feature_warning
0,loan_default,int64,70478,2,2.267000e-01,0.0,1.000000e+00,0,70478,0.000000,0.000028,NUMERIC,NO WARNING
1,ID,int64,70478,70478,5.027010e+05,20.0,1.005120e+06,0,70478,0.000000,1.000000,NUMERIC,"EXCLUDE, GT 90% UNIQUE"
2,Tax_Liens,float64,70144,11,2.880000e-02,0.0,1.100000e+01,334,70144,0.004739,0.000156,NUMERIC,NO WARNING
3,Bankruptcies,float64,69990,8,1.175000e-01,0.0,7.000000e+00,488,69990,0.006924,0.000114,NUMERIC,NO WARNING
4,Maximum_Open_Credit,float64,70150,37594,7.718260e+05,0.0,1.539738e+09,328,70150,0.004654,0.533415,NUMERIC,NO WARNING
5,Current_Credit_Balance,float64,70152,28816,2.944997e+05,0.0,1.298696e+07,326,70152,0.004626,0.408865,NUMERIC,NO WARNING
6,Number_of_Credit_Problems,float64,70152,13,1.672000e-01,0.0,1.200000e+01,326,70152,0.004626,0.000184,NUMERIC,NO WARNING
7,Number_of_Open_Accounts,float64,70152,51,1.114820e+01,0.0,7.600000e+01,326,70152,0.004626,0.000724,NUMERIC,NO WARNING
8,Months_since_last_delinquent,float64,32937,95,3.480020e+01,0.0,9.700000e+01,37541,32937,0.532663,0.001348,NUMERIC,"EXCLUDE, GT 50% MISSING"
9,Current_Loan_Amount,float64,70152,19782,1.165346e+07,10802.0,1.000000e+08,326,70152,0.004626,0.280683,NUMERIC,NO WARNING


## Extra Credit 2 - worth 10 pts 

using your data frame and df_profile results write a function that creates a histogram of NUMERIC by TARGET

use your funcion to visualize NUMERIC data in the following datasets 
    - churn.csv
    - bank-additional-full.csv
    - loans.csv